In [69]:
"""
Data from the current season fragment will be plugged into the model to get picks.
The most recent full season will be the test dataset.
All seasons before that will be the train dataset.
"""

import numpy as np
import pandas as pd
import datetime as dt
import http.client
import json
import time
from bs4 import BeautifulSoup
import joblib
import requests

pd.set_option('display.max_columns', 100)

# Define season we are currently betting 
#   (and the test dataset is the entire season before that one)
season_to_bet = 2024
season_to_test = season_to_bet - 1

# Define leagues we are currently betting
#   E0: the English Premier League
#   D1: the German Bundesliga
#   SP1: Spanish La Liga
leagues_of_choice = ["E0", "D1", "SP1"]

# Define years being used to train the model
years_of_choice = ["2425", "2324", "2223", 
                   "2122", "2021", "1920", "1819", 
                   "1718", "1617", "1516", "1415", 
                   "1314", "1213", "1112", "1011"]

# URLs to scrape for upcoming fixtures
url_dict = {
    "E0": "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures",
    "D1": "https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures",
    "SP1": "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"
}

# Set the days we will be checking for matches to bet
date_of_check = dt.date.today()
end_of_checked_week = dt.date.today() + dt.timedelta(days=7)
days_to_bet = pd.date_range(date_of_check, end_of_checked_week)
days_to_bet

DatetimeIndex(['2024-12-03', '2024-12-04', '2024-12-05', '2024-12-06',
               '2024-12-07', '2024-12-08', '2024-12-09', '2024-12-10'],
              dtype='datetime64[ns]', freq='D')

In [70]:
"""
Get data sets from football-data.co.uk
"""

all_matches_raw = pd.DataFrame()

for eachLeague in leagues_of_choice:
    for eachSeason in years_of_choice:
        time.sleep(5)
        url = f"https://www.football-data.co.uk/mmz4281/{eachSeason}/{eachLeague}.csv"
        try:
            temp_df = pd.read_csv(url)
            temp_year = eachSeason[0:2]
            temp_df['Season'] = int(f"20{temp_year}")
            all_matches_raw = pd.concat([all_matches_raw, temp_df], axis=0)
        except: 
            pass


In [71]:
"""
Target variables: home win vs away win
"""

all_matches_raw["homewin"] = (all_matches_raw['FTR'] == 'H').astype(int)
all_matches_raw["awaywin"] = (all_matches_raw['FTR'] == 'A').astype(int)

In [72]:
"""
Dates in the file are wonky. Make them consistent with 
the built-in Pandas date parser "to_datetime"
"""

all_matches_raw['FixedDate'] = pd.to_datetime(all_matches_raw['Date'], format='mixed', dayfirst=True)

all_matches_raw.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,BFH,BFD,BFA,PSH,PSD,PSA,WHH,WHD,WHA,1XBH,1XBD,1XBA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,BFEH,BFED,...,IWD,IWA,VCH,VCD,VCA,IWCH,IWCD,IWCA,VCCH,VCCD,VCCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA,SJH,SJD,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA,Unnamed: 70,Unnamed: 71,Unnamed: 72,homewin,awaywin,FixedDate
301,D1,14/05/11,NaN,Hamburg,M'gladbach,1.0,1.0,D,0.0,1.0,A,NaN,16.0,9.0,4.0,3.0,8.0,10.0,7.0,5.0,1.0,2.0,0.0,0.0,2.88,3.5,2.30,2.80,3.60,2.25,NaN,NaN,NaN,NaN,NaN,NaN,2.75,3.3,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,2.1,2.75,3.60,2.4,NaN,NaN,NaN,NaN,NaN,NaN,37.0,2.95,2.76,3.75,3.50,2.65,2.39,25.0,1.60,1.53,2.53,2.42,21.0,0.00,2.10,2.03,1.97,1.81,2.80,3.75,2.25,2.63,3.75,2.40,2.80,3.50,2.3,2.80,3.5,2.30,2.70,3.6,2.20,NaN,NaN,NaN,0,0,2011-05-14
302,D1,14/05/11,NaN,Hannover,Nurnberg,3.0,1.0,H,1.0,1.0,D,NaN,20.0,9.0,5.0,2.0,22.0,23.0,5.0,4.0,2.0,2.0,0.0,0.0,2.05,3.5,3.40,2.00,3.40,3.50,NaN,NaN,NaN,NaN,NaN,NaN,2.10,3.4,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.4,3.6,2.00,3.60,3.5,NaN,NaN,NaN,NaN,NaN,NaN,38.0,2.10,2.04,3.77,3.51,3.64,3.44,25.0,1.64,1.57,2.38,2.31,19.0,-0.50,2.11,2.05,1.86,1.82,2.05,3.50,3.40,1.95,3.75,3.60,2.00,3.50,3.4,2.05,3.5,3.40,1.95,3.5,3.30,NaN,NaN,NaN,1,0,2011-05-14
303,D1,14/05/11,NaN,Hoffenheim,Wolfsburg,1.0,3.0,A,0.0,0.0,D,NaN,17.0,20.0,4.0,6.0,11.0,17.0,7.0,7.0,0.0,1.0,0.0,0.0,3.50,3.6,2.00,3.60,3.60,1.90,NaN,NaN,NaN,NaN,NaN,NaN,3.40,3.4,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,1.8,3.40,3.75,2.0,NaN,NaN,NaN,NaN,NaN,NaN,37.0,3.75,3.43,3.90,3.56,2.27,2.02,25.0,1.62,1.51,2.55,2.45,17.0,0.50,1.86,1.80,2.28,2.02,3.75,3.60,1.91,3.25,3.75,2.05,3.50,3.75,1.9,3.60,3.6,1.91,3.30,3.5,1.95,NaN,NaN,NaN,0,1,2011-05-14
304,D1,14/05/11,NaN,Kaiserslautern,Werder Bremen,3.0,2.0,H,3.0,1.0,H,NaN,12.0,17.0,4.0,7.0,14.0,10.0,4.0,8.0,0.0,2.0,0.0,0.0,2.30,3.5,2.88,2.15,3.45,3.05,NaN,NaN,NaN,NaN,NaN,NaN,2.30,3.4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.4,2.8,2.25,3.50,3.0,NaN,NaN,NaN,NaN,NaN,NaN,38.0,2.30,2.21,3.75,3.43,3.47,3.11,25.0,1.53,1.46,2.75,2.59,19.0,-0.25,1.98,1.90,2.04,1.98,2.25,3.60,2.90,2.20,3.75,3.00,2.25,3.30,3.0,2.30,3.4,2.88,2.10,3.4,3.00,NaN,NaN,NaN,1,0,2011-05-14
305,D1,14/05/11,NaN,Mainz,St Pauli,2.0,1.0,H,0.0,1.0,A,NaN,19.0,14.0,7.0,4.0,11.0,13.0,7.0,3.0,0.0,2.0,0.0,0.0,1.36,5.0,7.50,1.33,5.25,7.25,NaN,NaN,NaN,NaN,NaN,NaN,1.36,4.8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.5,7.3,1.33,5.00,8.5,NaN,NaN,NaN,NaN,NaN,NaN,38.0,1.38,1.36,5.45,4.93,8.80,7.97,25.0,1.53,1.45,2.82,2.62,17.0,-1.50,2.08,2.02,1.89,1.83,1.36,5.00,7.50,1.36,5.00,8.00,1.35,5.00,7.5,1.36,4.5,8.50,1.33,4.8,7.00,NaN,NaN,NaN,1,0,2011-05-14


In [73]:
"""
Drop columns I won't be using

And make the others all lower case
"""

my_cols = ['Div', 'Season', 'FixedDate', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'homewin', 'awaywin', 
           'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF']

df_raw = all_matches_raw[my_cols] 

df_raw.columns = [c.lower() for c in df_raw.columns]

df_raw.head()

,div,season,fixeddate,hometeam,awayteam,fthg,ftag,homewin,awaywin,hs,as,hst,ast,hc,ac,hf,af
0,E0,2024,2024-08-16,Man United,Fulham,1.0,0.0,1,0,14.0,10.0,5.0,2.0,7.0,8.0,12.0,10.0
1,E0,2024,2024-08-17,Ipswich,Liverpool,0.0,2.0,0,1,7.0,18.0,2.0,5.0,2.0,10.0,9.0,18.0
2,E0,2024,2024-08-17,Arsenal,Wolves,2.0,0.0,1,0,18.0,9.0,6.0,3.0,8.0,2.0,17.0,14.0
3,E0,2024,2024-08-17,Everton,Brighton,0.0,3.0,0,1,9.0,10.0,1.0,5.0,1.0,5.0,8.0,8.0
4,E0,2024,2024-08-17,Newcastle,Southampton,1.0,0.0,1,0,3.0,19.0,1.0,4.0,3.0,12.0,15.0,16.0


In [74]:
"""
Make rolling averages for the last three matches
"""

def rolling_averages(group, cols, new_cols):
    group = group.sort_values("fixeddate")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [75]:
cols = ["hs", "as", "hst", "ast", "hc", "ac", "hf", "af"]
new_cols = [f"{c}_rolling" for c in cols]

raw_rolling = df_raw.groupby('hometeam').apply(lambda x: rolling_averages(x, cols, new_cols))

raw_rolling = raw_rolling.droplevel('hometeam')

raw_rolling = raw_rolling[raw_rolling['season'] < season_to_bet]

raw_rolling.head()

/var/folders/1_/vcjgnltn4xj_lflzdwk60wz80000gn/T/ipykernel_30084/367224742.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  raw_rolling = df_raw.groupby('hometeam').apply(lambda x: rolling_averages(x, cols, new_cols))


,div,season,fixeddate,hometeam,awayteam,fthg,ftag,homewin,awaywin,hs,as,hst,ast,hc,ac,hf,af,hs_rolling,as_rolling,hst_rolling,ast_rolling,hc_rolling,ac_rolling,hf_rolling,af_rolling
70,E0,2010,2010-10-16,Arsenal,Birmingham,2.0,1.0,1,0,18.0,6.0,13.0,4.0,7.0,3.0,16.0,9.0,24.333333,5.666667,14.000000,4.000000,11.000000,4.000000,11.666667,9.333333
90,E0,2010,2010-10-30,Arsenal,West Ham,1.0,0.0,1,0,20.0,6.0,13.0,3.0,7.0,4.0,14.0,18.0,21.666667,6.666667,13.000000,5.000000,10.666667,4.333333,14.000000,11.333333
107,E0,2010,2010-11-07,Arsenal,Newcastle,0.0,1.0,0,1,11.0,7.0,8.0,2.0,12.0,2.0,16.0,12.0,20.666667,6.333333,12.666667,4.000000,10.000000,3.666667,15.000000,13.666667
130,E0,2010,2010-11-20,Arsenal,Tottenham,2.0,3.0,0,1,12.0,10.0,7.0,6.0,2.0,6.0,17.0,16.0,16.333333,6.333333,11.333333,3.000000,8.666667,3.000000,15.333333,13.000000
150,E0,2010,2010-12-04,Arsenal,Fulham,2.0,1.0,1,0,15.0,11.0,9.0,7.0,3.0,7.0,7.0,5.0,14.333333,7.666667,9.333333,3.666667,7.000000,4.000000,15.666667,15.333333


In [76]:
"""
Get unique values in the "Div" column 

(We don't just use the list of league codes from above, as a way of checking the data)
"""

league_code_list = raw_rolling['div'].unique()

league_code_list

array(['E0', 'D1'], dtype=object)

In [77]:
# Make all historic data

for league in league_code_list:
    league_hist_df = raw_rolling[(raw_rolling['div'] == league) & (raw_rolling['season'] < season_to_bet)]
    league_hist_df.to_csv(f"data/{league}_2010to{season_to_bet - 1}.csv")

In [78]:
# Make data for current season that can be used to predict upcoming match results

for league in league_code_list:
    league_season_df = df_raw[(df_raw['div'] == league) & (df_raw['season'] == season_to_bet)]
    league_season_last3 = league_season_df.sort_values(["hometeam", "fixeddate"]).groupby('hometeam').tail(3)
    last3_stats = league_season_last3.groupby("hometeam")[cols].sum()
    last3_stats.columns = new_cols
    last3_stats[new_cols] = last3_stats[new_cols].sub(last3_stats[new_cols].mean(axis=0)).div(last3_stats[new_cols].std(axis=0))
    league_season_sum = league_season_df.groupby('hometeam')[cols].sum()
    league_season_sum[cols] = league_season_sum[cols].sub(league_season_sum[cols].mean(axis=0)).div(league_season_sum[cols].std(axis=0))
    league_season_sum[new_cols] = last3_stats
    league_season_sum.to_csv(f"data/{league}_{season_to_bet}.csv")
    

In [79]:
"""
Function to make models

I liked the XGBoosted model, but it was picking very strange upsets
and may have been overfitted to the training data.
So I'm going back to my previous model, Logistic Regression
"""

from sklearn.linear_model import LogisticRegression

def make_models(leagueName):
    # Get data, isolate relevant rows/columns for modeling
    matches = pd.read_csv(f"data/{leagueName}_2010to{season_to_test}.csv", index_col=0)
    train_df = matches[matches["season"] < season_to_test]
    test_df = matches[matches["season"] == season_to_test]
    targets = ["homewin", "awaywin"]
    predictors = ["hs", "as", "hst", "ast", "hc", "ac", "hf", "af",
              "hs_rolling", "as_rolling", "hst_rolling", "ast_rolling",
              "hc_rolling", "ac_rolling", "hf_rolling", "af_rolling"]
    all_vars = targets + predictors
    train_df = train_df[all_vars]
    test_df = test_df[all_vars]
    
    # Data transforms: Normalize and fill NA with 0
    train_df[predictors] = train_df[predictors]\
    .sub(train_df[predictors].mean(axis=0))\
    .div(train_df[predictors].std(axis=0))
    
    test_df[predictors] = test_df[predictors]\
    .sub(test_df[predictors].mean(axis=0))\
    .div(test_df[predictors].std(axis=0))
    
    train_df = train_df.fillna(0)
    test_df = test_df.fillna(0)
    
    # First, predict a home win
    lr_home = LogisticRegression(C=0.01, solver='liblinear')
    lr_home.fit(train_df[predictors], train_df["homewin"])
    joblib.dump(lr_home, f'models/{leagueName}_homewin.pkl')
    
    # Next, predict away win
    lr_away = LogisticRegression(C=0.01, solver='liblinear')
    lr_away.fit(train_df[predictors], train_df["awaywin"])
    joblib.dump(lr_away, f'models/{leagueName}_awaywin.pkl')

In [80]:
"""
Run the function
"""

for league in leagues_of_choice:
    make_models(league)

In [81]:
"""
Define arrays of column names that we will need
after scraping the upcoming matches
"""

cols_to_keep = ['Wk', 'Day', 'Date', 'Home', 'Away']

model_cols = ['hs', 'as', 'hst', 'ast', 'hc', 'ac', 'hf', 'af', 
              'hs_rolling', 'as_rolling', 'hst_rolling', 'ast_rolling', 
              'hc_rolling', 'ac_rolling', 'hf_rolling', 'af_rolling']

In [82]:
"""
Function to scrape the week's matches
"""

from io import StringIO

def get_matches(theLeague, theSeason):
    # Get upcoming fixtures
    theurl = url_dict[theLeague]
    data = requests.get(theurl)
    thisfix = pd.read_html(StringIO(str(data.text)))[0]
    thisfix['Date'] = pd.to_datetime(thisfix['Date'])
    thisfix = thisfix[thisfix['Date'].isin(days_to_bet)]
    thisfix = thisfix[cols_to_keep]
    
    # Attach predictive data from this season to those fixtures
    season_to_date = pd.read_csv(f"data/{theLeague}_{theSeason}.csv")
    
    # Need to execute a join, but names are not consistent across files
    names1 = list(set(np.append(thisfix['Home'].unique(), thisfix['Away'].unique())))
    names1.sort()
    season_to_date['newhome'] = names1
    matches_to_bet = thisfix.merge(season_to_date, how='left', left_on ='Home', right_on='newhome')
    
    return matches_to_bet

In [83]:
"""
Function to apply our models to the dataframe we just scraped
"""

def apply_models(theLeague, theDF):
    # Apply model for home wins
    homewin_model = joblib.load(f'models/{theLeague}_homewin.pkl')
    theDF['pred_homewin'] = homewin_model.predict(theDF[model_cols])
    homewin_probs = homewin_model.predict_proba(theDF[model_cols])
    theDF['prob_homewin'] = homewin_probs[:, 1]
    
    # Apply model for away wins
    awaywin_model = joblib.load(f'models/{theLeague}_awaywin.pkl')
    theDF['pred_awaywin'] = awaywin_model.predict(theDF[model_cols])
    awaywin_probs = awaywin_model.predict_proba(theDF[model_cols])
    theDF['prob_awaywin'] = awaywin_probs[:, 1]
    
    theDF['prob_draw'] = 1 - theDF['prob_homewin'] - theDF['prob_awaywin']
    theDF['prob_diff'] = np.abs(theDF['prob_homewin'] - theDF['prob_awaywin'])
    theDF['draw_threat'] = np.where(theDF['prob_diff'] < 0.1, 1, 
                            np.where(theDF['prob_draw'] > 0.25, 1, 0))
    theDF['div'] = theLeague
    
    betslip = theDF[['div', 'Wk', 'Day', 'Home', 'Away', 
                     'pred_homewin', 'prob_homewin',
                     'pred_awaywin', 'prob_awaywin',
                     'prob_draw', 'draw_threat',
                    ]]
    return betslip

In [84]:
"""
Bring the two functions together
"""

def make_betslip(leagueList, aYear):
    betslip = pd.DataFrame()
    for league in leagueList:
        lg_df = get_matches(league, aYear)
        lg_betslip = apply_models(league, lg_df)
        betslip = pd.concat([betslip, lg_betslip], axis=0)
    return betslip

In [85]:
"""
This week's betslip
"""

make_betslip(leagues_of_choice, season_to_bet)

,Wk,Day,Home,Away,pred_homewin,prob_homewin,pred_awaywin,prob_awaywin,prob_draw,draw_threat
0,14.0,Tue,Ipswich Town,Crystal Palace,0,0.161505,1,0.591255,0.247240,0
1,14.0,Tue,Leicester City,West Ham,0,0.097680,1,0.772706,0.129614,0
2,14.0,Wed,Everton,Wolves,0,0.409425,0,0.406961,0.183615,1
3,14.0,Wed,Newcastle Utd,Liverpool,0,0.265206,0,0.462211,0.272584,1
4,14.0,Wed,Manchester City,Nott'ham Forest,1,0.690891,0,0.126411,0.182699,0
5,14.0,Wed,Southampton,Chelsea,0,0.066376,1,0.826464,0.107161,0
6,14.0,Wed,Arsenal,Manchester Utd,1,0.843497,0,0.063351,0.093152,0
7,14.0,Wed,Aston Villa,Brentford,0,0.435691,0,0.290270,0.274039,1
8,14.0,Thu,Fulham,Brighton,1,0.693478,0,0.127263,0.179259,0
9,14.0,Thu,Bournemouth,Tottenham,1,0.627578,0,0.164874,0.207548,0
